# Obesity Classification Model Training and Evaluation Notebook
This notebook handles:
- Model Training
- Hyper-Parameter Tuning
- Model Evaluation

In [1]:
import pandas as pd
from narwhals import Boolean

X_train = pd.read_csv("data/preprocessed data/X_train.csv")
X_val = pd.read_csv("data/preprocessed data/X_val.csv")
X_test = pd.read_csv("data/preprocessed data/X_test.csv")
y_train = pd.read_csv("data/preprocessed data/y_train.csv", header=None).to_numpy().ravel()
y_val = pd.read_csv("data/preprocessed data/y_val.csv", header=None).to_numpy().ravel()
y_test = pd.read_csv("data/preprocessed data/y_test.csv", header=None).to_numpy().ravel()
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((1251, 21), (418, 21), (418, 21), (1251,), (418,), (418,))

## Model Evaluation

In [2]:
model_evaluations = pd.DataFrame(
    columns=["Train_Accuracy", "Val_Accuracy", "Test_Accuracy", "Balanced_Accuracy", "Precision", "Recall", "F1_Score"])
model_evaluations.index.name = "Model"
model_evaluations.head()

,Train_Accuracy,Val_Accuracy,Test_Accuracy,Balanced_Accuracy,Precision,Recall,F1_Score
Model,,,,,,,


In [3]:
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
import joblib


def evaluate_model(model_name: str, model: BaseEstimator) -> None:
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)
    model_evaluations.loc[model_name, "Train_Accuracy"] = accuracy_score(y_train, train_pred)
    model_evaluations.loc[model_name, "Val_Accuracy"] = accuracy_score(y_val, val_pred)
    model_evaluations.loc[model_name, "Test_Accuracy"] = accuracy_score(y_test, test_pred)
    model_evaluations.loc[model_name, "Balanced_Accuracy"] = balanced_accuracy_score(y_test, test_pred)
    model_evaluations.loc[model_name, "Precision"] = precision_score(y_test, test_pred, average="weighted",
                                                                     zero_division=0)
    model_evaluations.loc[model_name, "Recall"] = recall_score(y_test, test_pred, average="weighted", zero_division=0)
    model_evaluations.loc[model_name, "F1_Score"] = f1_score(y_test, test_pred, average="weighted", zero_division=0)
    joblib.dump(model, f"models/{model_name}.pkl")
    return model_evaluations.loc[model_name]

## GridSearch Cross-Validation Hyper-Parameter Tuning

In [4]:
from sklearn.model_selection import GridSearchCV


def hyper_parameter_tune(model: BaseEstimator, param_grid: dict, verbose: bool = True) -> BaseEstimator:
    grid_search = GridSearchCV(model, param_grid, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    if verbose:
        print(grid_search.best_params_)
    return grid_search.best_estimator_

## Model Training

### Gaussian Naive Bayes Classifier

In [5]:
from sklearn.naive_bayes import GaussianNB

GaussianNB_model = GaussianNB()
params = {"var_smoothing": [1e-9, 1e-6, 1e-3, 1e-2, 1e-1]}
GaussianNB_model = hyper_parameter_tune(GaussianNB_model, params)
evaluate_model("GaussianNB", GaussianNB_model)

{'var_smoothing': 0.01}


Train_Accuracy       0.821743
Val_Accuracy         0.794258
Test_Accuracy        0.811005
Balanced_Accuracy    0.810421
Precision            0.806936
Recall               0.811005
F1_Score              0.80523
Name: GaussianNB, dtype: object

### Logistic Regression Classifier

In [6]:
from sklearn.linear_model import LogisticRegression

LogisticRegression_model = LogisticRegression(random_state=42, n_jobs=-1)
params = {"C": [1.0, 0.1, 10]}
LogisticRegression_model = hyper_parameter_tune(LogisticRegression_model, params)
evaluate_model("LogisticRegression", LogisticRegression_model)

{'C': 10}


Train_Accuracy        0.96243
Val_Accuracy         0.949761
Test_Accuracy        0.956938
Balanced_Accuracy    0.955753
Precision            0.957775
Recall               0.956938
F1_Score             0.956974
Name: LogisticRegression, dtype: object

### K-Nearest Neighbors Classifier

In [7]:
from sklearn.neighbors import KNeighborsClassifier

KNeighborsClassifier_model = KNeighborsClassifier(n_jobs=-1)
params = {"n_neighbors": [3, 5, 7]}
KNeighborsClassifier_model = hyper_parameter_tune(KNeighborsClassifier_model, params)
evaluate_model("KNeighborsClassifier", KNeighborsClassifier_model)

{'n_neighbors': 3}


Train_Accuracy       0.930456
Val_Accuracy         0.892344
Test_Accuracy         0.84689
Balanced_Accuracy    0.840845
Precision            0.848875
Recall                0.84689
F1_Score             0.838273
Name: KNeighborsClassifier, dtype: object

### Decision Tree Classifier

In [8]:
from sklearn.tree import DecisionTreeClassifier

DecisionTreeClassifier_model = DecisionTreeClassifier(random_state=42)
params = {"max_depth": [3, 5, 7]}
DecisionTreeClassifier_model = hyper_parameter_tune(DecisionTreeClassifier_model, params)
evaluate_model("DecisionTreeClassifier", DecisionTreeClassifier_model)

{'max_depth': 7}


Train_Accuracy            1.0
Val_Accuracy         0.976077
Test_Accuracy        0.978469
Balanced_Accuracy    0.976967
Precision            0.978897
Recall               0.978469
F1_Score             0.978374
Name: DecisionTreeClassifier, dtype: object

### Support Vector Classifier

In [9]:
from sklearn.svm import SVC

SVC_model = SVC(random_state=42)
params = {"kernel": ["linear", "poly", "rbf"], "C": [1.0, 0.5, 1.5]}
SVC_model = hyper_parameter_tune(SVC_model, params)
evaluate_model("SVC", SVC_model)

{'C': 1.5, 'kernel': 'linear'}


Train_Accuracy       0.972022
Val_Accuracy         0.954545
Test_Accuracy        0.961722
Balanced_Accuracy    0.960664
Precision            0.962115
Recall               0.961722
F1_Score             0.961684
Name: SVC, dtype: object

### Random Forest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier_model = RandomForestClassifier(max_depth=5, random_state=42, n_jobs=-1)
params = {"ccp_alpha": [0.01, 0.09, 0.011]}
RandomForestClassifier_model = hyper_parameter_tune(RandomForestClassifier_model, params)
evaluate_model("RandomForestClassifier", RandomForestClassifier_model)

{'ccp_alpha': 0.011}


Train_Accuracy       0.960831
Val_Accuracy         0.956938
Test_Accuracy        0.947368
Balanced_Accuracy    0.944758
Precision            0.947893
Recall               0.947368
F1_Score             0.946929
Name: RandomForestClassifier, dtype: object

### Gradient Boosting Classifier

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

GradientBoostingClassifier_model = GradientBoostingClassifier(random_state=42)
params = {"learning_rate": [0.1, 0.09, 0.11], "ccp_alpha": [0.01, 0.09, 0.11]}
GradientBoostingClassifier_model = hyper_parameter_tune(GradientBoostingClassifier_model, params)
evaluate_model("GradientBoostingClassifier", GradientBoostingClassifier_model)

{'ccp_alpha': 0.01, 'learning_rate': 0.1}


Train_Accuracy       0.965627
Val_Accuracy         0.961722
Test_Accuracy         0.95933
Balanced_Accuracy    0.958763
Precision             0.96065
Recall                0.95933
F1_Score             0.959117
Name: GradientBoostingClassifier, dtype: object

### Multilayer Perceptron Classifier

In [12]:
from sklearn.neural_network import MLPClassifier

MLPClassifier_model = MLPClassifier(early_stopping=True, random_state=42)
params = {"hidden_layer_sizes": [(100,), (50, 50), (25, 10, 5)]}
MLPClassifier_model = hyper_parameter_tune(MLPClassifier_model, params)
evaluate_model("MLPClassifier", MLPClassifier_model)

{'hidden_layer_sizes': (50, 50)}


Train_Accuracy       0.964828
Val_Accuracy         0.913876
Test_Accuracy        0.937799
Balanced_Accuracy      0.9355
Precision            0.937798
Recall               0.937799
F1_Score             0.937682
Name: MLPClassifier, dtype: object

In [13]:
model_evaluations

,Train_Accuracy,Val_Accuracy,Test_Accuracy,Balanced_Accuracy,Precision,Recall,F1_Score
Model,,,,,,,
GaussianNB,0.821743,0.794258,0.811005,0.810421,0.806936,0.811005,0.80523
LogisticRegression,0.96243,0.949761,0.956938,0.955753,0.957775,0.956938,0.956974
KNeighborsClassifier,0.930456,0.892344,0.84689,0.840845,0.848875,0.84689,0.838273
DecisionTreeClassifier,1.0,0.976077,0.978469,0.976967,0.978897,0.978469,0.978374
SVC,0.972022,0.954545,0.961722,0.960664,0.962115,0.961722,0.961684
RandomForestClassifier,0.960831,0.956938,0.947368,0.944758,0.947893,0.947368,0.946929
GradientBoostingClassifier,0.965627,0.961722,0.95933,0.958763,0.96065,0.95933,0.959117
MLPClassifier,0.964828,0.913876,0.937799,0.9355,0.937798,0.937799,0.937682
